# Estimation of steer anngle from odometry measurements
## Experiments: 2018-07-03<br>
Erik Frisk (erik.frisk@liu.se)<br>
Department of Electrical Engineering<br>
Linköping University<br>
Sweden

In [ ]:
import sys
if not ('../python' in sys.path):
    sys.path.append('../python')

import matplotlib.pyplot as plt
from calib_util import LoadLogDataPickle, BoxOff
from calib_util import QualisysSpeed, OdoSpeed, GetCalibrationSection
import numpy as np
import pandas as pd
from scipy.optimize import minimize

datadir = '../cascar_logs/20180703/'

In [ ]:
%matplotlib 

#from pylab import rcParams
#%matplotlib inline
#rcParams['figure.figsize']=np.array(rcParams['figure.figsize'])*1.8

# Load and preprocess data

Let the estimated angular velocity from both left and right be equal
$$
\frac{\frac{2\pi r}{10}}{\Delta T_L}\frac{1}{R-\frac{W}{2}} = \frac{\frac{2\pi r}{10}}{\Delta T_R}\frac{1}{R+\frac{W}{2}}
$$
then the estimated current radius is
$$
R = \frac{\Delta T_L + \Delta T_R}{\Delta T_L - \Delta T_R}\frac{W}{2}
$$
which then gives an estimate of the steering angle
$$
\tan(\delta) = \frac{L}{R} = \frac{\Delta T_L - \Delta T_R}{\Delta T_L + \Delta T_R}\frac{L}{\frac{W}{2}} = \frac{1 - c}{1 + c}\frac{L}{\frac{W}{2}}
$$
where
$$
c = \frac{\Delta T_R}{\Delta T_L}
$$

In [ ]:
def SteerFunction(steer, theta):
    return theta[0] + steer*theta[1]

def SteerEstimate(dt_l, dt_r, theta):
    L = theta[3]
    W = theta[4]
    c = dt_r/dt_l
    return np.arctan((1-c)/(1+c)/(W/2)*L)

In [ ]:
tau = 270/1000

d = {}
pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_13_58_slow.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=False, synchronize=False)
odo_l['steer'] = np.interp(odo_l['t'], odo_l['t']+tau, odo_l['steer'])
odo_r['steer'] = np.interp(odo_r['t'], odo_r['t']+tau, odo_r['steer'])
d['slow'] = (pos, odo_l, odo_r)

pos, odo_l, odo_r = LoadLogDataPickle(datadir + 'cascar_201873_14_1_fast.pickle', 
                                      body='cascar', n_skip=1, 
                                      calibrate_phi=False, synchronize=False)
odo_l['steer'] = np.interp(odo_l['t'], odo_l['t']+tau, odo_l['steer'])
odo_r['steer'] = np.interp(odo_r['t'], odo_r['t']+tau, odo_r['steer'])
d['fast'] = (pos, odo_l, odo_r)

theta = [0.01941286, 0.00213721, 0.04131876, 0.17626054, 0.20801631]

# Analyze different estimates of delta

In [ ]:
for k, ds in enumerate(d.keys()):
    (_, odo_l, odo_r) = d[ds]
    td_hat = SteerEstimate(odo_l.dt, odo_r.dt, theta)
    td_hat_l = SteerFunction(odo_l.steer, theta)
    td_hat_r = SteerFunction(odo_r.steer, theta)

    plt.figure(10 + k, clear=True)

    ax1 = plt.subplot(3, 1, 1)
    plt.plot(odo_l.t, td_hat*180/np.pi, 'b', label='time based estimate')
    plt.plot(odo_l.t, td_hat_l*180/np.pi, 'r', label='steer function left')
    plt.legend(loc='upper right')
    plt.xlabel('t [s]')
    plt.ylabel('degrees')
    plt.title('delta estimates (' + ds + ')')
    BoxOff()

    plt.subplot(3, 1, 2, sharex=ax1)
    plt.plot(odo_l.t, odo_l.vel, 'b', label='Velocity')
    plt.plot(odo_l.t, odo_l.steer, 'r', label='Steer')
    plt.xlabel('t [s]')
    plt.ylabel('%')
    plt.title('Velocity and steer')
    plt.legend(loc='upper right')
    BoxOff()

    plt.subplot(3, 1, 3, sharex=ax1)
    plt.plot(odo_l.t, odo_l.dt*1000, 'b', label='left')
    plt.plot(odo_l.t, odo_r.dt*1000, 'r', label='right')
    plt.xlabel('t [s]')
    plt.ylabel('ms')
    plt.title('Delta time, left and right')
    plt.ylim(0, np.median(odo_l.dt)*1000*2)
    plt.legend(loc='upper right')
    BoxOff()
    plt.tight_layout()
